## Description

This sample Jupyter notebook is designed to demonstrate how to read the first page of the user Favorites list for this account. The list ios added to a Pandas structure where it can be easily manipulated. You get a list of exchanges and symbols that can be used in the other sample reports.

You can click here to log into your Monitor+ Favorites portal where you can manage your favorite datasets and datasources.



## Imports

In [ ]:
# from class_source.idata_client import IData
from API.monitorplus import IData
import pandas as pd
import numpy as np
import colorama
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

: 

## Global Variables

In [ ]:
# Customers API Key to use in request for data from the Monitor+ API
API_KEY = "VOFWM-43RCS-PUH5T-A0X8I"

# Datasets to use  in API
# Datasource1 = "ECBFX"
# DS_Symbol1 = "EURGBP"
API_Datasource1 = "IEO-CDSL"
API_DSSymbol1 = "AUDUSD"
# Name is filled from GetValuesRC if request is successful
DS_Name1 = ""

# Data Scurce2 details
# Datasource2 = "ECBFX"
# DS_Symbol2 = "EURUSD"
API_Datasource2 = "IEO-CDSL"
API_DSSymbol2 = "CADUSD"
# Name is filled from GetValuesRC if request is successful
DS_Name2 = ""

    

## API initialisation Code

In [ ]:
# initialise the Python class
idata = IData()

#Store the API Key in the Python Class
idata.set_api_key(API_KEY)

# We will retuen the metadata for the user favorite datasets
result = idata.get_user_favorites()
if (result is None):
    print("Could not get the user datasources. Check terminal for errors.")
else:
    print("Metadata for 50 User datasets was downloaded successfully.")

print("")
    

## API request market data for the 2 datasets

The paramters to add to the API call are specified <a href="https://www.sarus.com/api-docs#getdatasetvaluesrc">here</a>

In [ ]:
#  print(result)
# Print  list of datasources, dataCategoriesm  symbols, descriptions,  startDate, end date, price count 

RC_Result = idata.get_dataset_values_rc(
[
    # you can change the datasource sand symbols to use in the Global Variable section above
    {"Datasource": Datasource1,"Symbol": DS_Symbol1, "HandleWeekends":"7f"},
    {"Datasource": Datasource2,"Symbol": DS_Symbol2, "HandleWeekends":"7f"}
]    
)   

df = pd.DataFrame(RC_Result)

# If success set the neme(s) as we use them to check if request was  successful.
DS_Name1 = ""
DS_Name2 = ""
print(df)

# Functions that help us to access the data.


In [ ]:
def get_daily_data(dictionary):

    columns = dictionary['Columns']
    rows = dictionary['Rows']
    corrections = dictionary['Corrections']
    df = pd.DataFrame(rows.values(), columns = [columns[0]['Name'], columns[1]['Name']], index = rows.keys())
    df = df.reset_index()
    df.rename(columns = {'index':'Date'}, inplace = True)
    df.dropna(axis = 0, inplace = True)
    df["Date"] = pd.to_datetime(df['Date'])
    df.insert(1, 'Year', df.Date.dt.year)
    df.insert(2, 'Month', df.Date.dt.month)

    return df

def get_averages(df):
    MthList = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    MthList_upd = {i+1:word for i,word in enumerate(MthList)}

    df = df.groupby(['Year', 'Month'], as_index= False).agg({"ECB - Euro vs Pound Sterling": np.mean, "ECB - Euro vs US Dollar":np.mean})
    df['Month'] = df['Month'].replace(MthList_upd)
    df.rename(columns = {"ECB - Euro vs Pound Sterling": "ECB - Euro vs Pound Sterling Mean", "ECB - Euro vs US Dollar": "ECB - Euro vs US Dollar Mean"}, inplace = True)
    return df

def get_processed(df):
    MthList = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    eurgbp = df.iloc[:, :-1]
    eurusd = df.drop(columns = ['ECB - Euro vs Pound Sterling Mean'], axis = 1)
    datas = [eurgbp, eurusd]
    for num, data in enumerate(datas):    
        temp = []
        for i in range(0, len(data), 12):
            lists = data.iloc[i:i+12, -1].values.tolist()
            temp.append(lists)
        datas[num] = pd.DataFrame(temp, columns = MthList, index = data.Year.unique().tolist()).round(3)
    datas = tuple(datas)
    
    return datas

# Retrieving the data into Pandas.

In [ ]:
df = get_daily_data(RC_Result)
Series1Name, Series2Name = df.columns[-2:]
df.tail(12)

The methods we will use require our data to be a Time-Series. That means the index of our Data-Frame has to be of type datetime. Also our data is recorded on bussiness days only, so we set frequency to "b", which means bussiness days.

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date", inplace=True)
df = df.asfreq('b')
df = df.fillna(method='ffill')

Plotting the two Time-Series.

In [ ]:
df[Series1Name].plot(figsize=(20,5), title = Series1Name)
df[Series2Name].plot(figsize=(20,5), title = Series2Name)
plt.title(f"{Series1Name} and {Series2Name}")
plt.show()

Plotting monthly data.

In [ ]:
df[Series1Name].asfreq("D")\
               .fillna(method="ffill").asfreq("m")\
               .plot(figsize=(20,5), title = Series1Name)
df[Series2Name].asfreq("D")\
               .fillna(method="ffill").asfreq("m")\
               .plot(figsize=(20,5), title = Series2Name)
plt.title("Euro vs Pound Sterling vs Euro vs US Dollar")
plt.show()

We will plot the quantile-quantile (Q-Q) plot.

In [ ]:
import scipy.stats
import pylab

In [ ]:
scipy.stats.probplot(df[Series1Name], plot = pylab)
plt.title(f"{Series1Name} QQ Plot", size = 24)
pylab.show()

In [ ]:
scipy.stats.probplot(df[Series2Name], plot = pylab)
plt.title(f"{Series2Name} QQ Plot", size = 24)
pylab.show()

The x-axis indicates the number of standard deviations from the mean. If the dots fit the straight line perfectly it means that the data is normally distributed. Most Time-Series are not normally distributed. The Time-Series that are normally distributed are called white-noise and are useless for most models. Hence, it is good that are Time